In [1]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('shops.csv', sep=',')  #shops.csv: 소상공인시장진흥공단_상가(상권)정보_서울

columns = ['상호명', '상권업종대분류명','상권업종소분류명', 
           '시도명', '시군구명', '행정동명', '도로명주소', 
           '경도', '위도']
df=df[columns].copy()

df = df.loc[(df['시군구명'] == '서대문구')]  #서대문구에 있는 음식점만

df_ = df.loc[df.상권업종대분류명 =="음식"]  #음식점만

dong=list(sorted(set(list((df_['행정동명']))))) #set: 자료형
len(dong)

dong_list=[]
for i in range(len(dong)): 
    dong_list.append(df_.loc[df.행정동명==dong[i]]) #행정동명으로 덩어리를 분류해  dong_list에 저장
    i+=1
dong_list[0]

In [15]:
!pip install selenium

from selenium import webdriver
chrome_options=webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-setuid-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_experimental_option('excludeSwitches',['enable-logging'])

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time

chromedriver = 'C:/Users/ChaeWon/Documents/3-1/파이썬과데이터분석/crawling/chromedriver.exe' 
driver = webdriver.Chrome(chromedriver, chrome_options=chrome_options) 

<ipython-input-15-bc2f2a44ecea>:22: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver, chrome_options=chrome_options)
<ipython-input-15-bc2f2a44ecea>:22: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chromedriver, chrome_options=chrome_options)


In [3]:
dong_list[0]['naver_keyword'] = dong_list[0]['행정동명']+'%20'+dong_list[0]['상호명']
dong_list[0]['naver_map_url'] = ''

# url 가져오기

for i, keyword in enumerate(dong_list[0]['naver_keyword'].tolist()):
    print("이번에 찾을 키워드 :", i, f"/ {dong_list[0].shape[0] -1} 행", keyword)
    try:
        naver_map_search_url = f"https://m.map.naver.com/search2/search.naver?query={keyword}&sm=hty&style=v5"
        driver.get(naver_map_search_url)
        
        time.sleep(3.5)
        dong_list[0].iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')

        #검색결과 없는 경우
    except Exception as e1:
        if "li:nth-child(1)" in str(e1):
            try:
                dong_list[0].iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
                time.sleep(1)
            except Exception as e2:
                print(e2)
                dong_list[0].iloc[i,-1] = np.nan
                time.sleep(1)
        else:
            pass


driver.quit()


# url 만들어주기
dong_list[0]['naver_map_url'] = "https://m.place.naver.com/restaurant/" + dong_list[0]['naver_map_url']


# URL이 수집되지 않은 데이터는 제거합니다.
dong_list[0] = dong_list[0].loc[~dong_list[0]['naver_map_url'].isnull()]

<ipython-input-3-72717b6437bf>:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)
<ipython-input-3-72717b6437bf>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dong_list[0]['naver_keyword'] = dong_list[0]['행정동명']+'%20'+dong_list[0]['상호명']
<ipython-input-3-72717b6437bf>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dong_list[0]['naver_map_url'] = ''


이번에 찾을 키워드 : 0 / 115 행 남가좌1동%20감자바우곤드레밥


<ipython-input-3-72717b6437bf>:25: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  dong_list[0].iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
C:\Users\ChaeWon\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


이번에 찾을 키워드 : 1 / 115 행 남가좌1동%20단비
이번에 찾을 키워드 : 2 / 115 행 남가좌1동%20고봉민김밥인
이번에 찾을 키워드 : 3 / 115 행 남가좌1동%20광풍정육식당
이번에 찾을 키워드 : 4 / 115 행 남가좌1동%20엄마맛집
이번에 찾을 키워드 : 5 / 115 행 남가좌1동%20안뜰


<ipython-input-3-72717b6437bf>:31: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  dong_list[0].iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')


Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=101.0.4951.54)
Stacktrace:
Backtrace:
	Ordinal0 [0x00767413+2389011]
	Ordinal0 [0x006F9F61+1941345]
	Ordinal0 [0x005EC658+837208]
	Ordinal0 [0x006191DD+1020381]
	Ordinal0 [0x0061949B+1021083]
	Ordinal0 [0x00646032+1204274]
	Ordinal0 [0x00634194+1130900]
	Ordinal0 [0x00644302+1196802]
	Ordinal0 [0x00633F66+1130342]
	Ordinal0 [0x0060E546+976198]
	Ordinal0 [0x0060F456+980054]
	GetHandleVerifier [0x00919632+1727522]
	GetHandleVerifier [0x009CBA4D+2457661]
	GetHandleVerifier [0x007FEB81+569713]
	GetHandleVerifier [0x007FDD76+566118]
	Ordinal0 [0x00700B2B+1968939]
	Ordinal0 [0x00705988+1989000]
	Ordinal0 [0x00705A75+1989237]
	Ordinal0 [0x0070ECB1+2026673]
	BaseThreadInitThunk [0x76AF7BA9+25]
	RtlInitializeExceptionChain [0x77CBB7DB+107]
	RtlClearBits [0x77CB

이번에 찾을 키워드 : 35 / 115 행 남가좌1동%20연우식당
이번에 찾을 키워드 : 36 / 115 행 남가좌1동%20모래내1번지
이번에 찾을 키워드 : 37 / 115 행 남가좌1동%20퀴즈노스
이번에 찾을 키워드 : 38 / 115 행 남가좌1동%20호수
이번에 찾을 키워드 : 39 / 115 행 남가좌1동%20원할머니보쌈
이번에 찾을 키워드 : 40 / 115 행 남가좌1동%20만나꼬마김밥
이번에 찾을 키워드 : 41 / 115 행 남가좌1동%20끝집
이번에 찾을 키워드 : 42 / 115 행 남가좌1동%20종로떡집
이번에 찾을 키워드 : 43 / 115 행 남가좌1동%20신참떡볶이
이번에 찾을 키워드 : 44 / 115 행 남가좌1동%20금돼지
이번에 찾을 키워드 : 45 / 115 행 남가좌1동%20발로만푸드
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=101.0.4951.54)
Stacktrace:
Backtrace:
	Ordinal0 [0x00767413+2389011]
	Ordinal0 [0x006F9F61+1941345]
	Ordinal0 [0x005EC658+837208]
	Ordinal0 [0x006191DD+1020381]
	Ordinal0 [0x0061949B+1021083]
	Ordinal0 [0x00646032+1204274]
	Ordinal0 [0x00634194+1130900]
	Ordinal0 [0x00644302+1196802]
	Ordinal0 [0x00633F66+1130342]
	Ordinal0 [0x0060E546+976198]
	Ordin

이번에 찾을 키워드 : 67 / 115 행 남가좌1동%20어제
이번에 찾을 키워드 : 68 / 115 행 남가좌1동%20대박집
이번에 찾을 키워드 : 69 / 115 행 남가좌1동%20이종구낙지세상
이번에 찾을 키워드 : 70 / 115 행 남가좌1동%20월드컵
이번에 찾을 키워드 : 71 / 115 행 남가좌1동%20캐슬
이번에 찾을 키워드 : 72 / 115 행 남가좌1동%20다같이카페
이번에 찾을 키워드 : 73 / 115 행 남가좌1동%20제일식당
이번에 찾을 키워드 : 74 / 115 행 남가좌1동%20구포갈비
이번에 찾을 키워드 : 75 / 115 행 남가좌1동%20로즈
이번에 찾을 키워드 : 76 / 115 행 남가좌1동%20왕세숫대야냉면24시해장국
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=101.0.4951.54)
Stacktrace:
Backtrace:
	Ordinal0 [0x00767413+2389011]
	Ordinal0 [0x006F9F61+1941345]
	Ordinal0 [0x005EC658+837208]
	Ordinal0 [0x006191DD+1020381]
	Ordinal0 [0x0061949B+1021083]
	Ordinal0 [0x00646032+1204274]
	Ordinal0 [0x00634194+1130900]
	Ordinal0 [0x00644302+1196802]
	Ordinal0 [0x00633F66+1130342]
	Ordinal0 [0x0060E546+976198]
	Ordinal0 [0x0060F456+980054]
	GetHandleV

이번에 찾을 키워드 : 115 / 115 행 남가좌1동%20싱싱회포장쎈터


<ipython-input-3-72717b6437bf>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dong_list[0]['naver_map_url'] = "https://m.place.naver.com/restaurant/" + dong_list[0]['naver_map_url']


In [7]:
dong_list[0].head()

,상호명,상권업종대분류명,상권업종소분류명,시도명,시군구명,행정동명,도로명주소,경도,위도,naver_keyword,naver_map_url
2634,감자바우곤드레밥,음식,한식/백반/한정식,서울특별시,서대문구,남가좌1동,서울특별시 서대문구 수색로2길 7,126.915967,37.569179,남가좌1동%20감자바우곤드레밥,https://m.place.naver.com/restaurant/13270081
7709,단비,음식,호프/맥주,서울특별시,서대문구,남가좌1동,서울특별시 서대문구 수색로 42,126.914240,37.570433,남가좌1동%20단비,https://m.place.naver.com/restaurant/1575351305
8090,고봉민김밥인,음식,라면김밥분식,서울특별시,서대문구,남가좌1동,서울특별시 서대문구 가재울미래로 2,126.917828,37.574368,남가좌1동%20고봉민김밥인,https://m.place.naver.com/restaurant/37793363
14688,광풍정육식당,음식,갈비/삼겹살,서울특별시,서대문구,남가좌1동,서울특별시 서대문구 수색로4가길 14-4,126.914553,37.570671,남가좌1동%20광풍정육식당,https://m.place.naver.com/restaurant/32731448
20094,엄마맛집,음식,한식/백반/한정식,서울특별시,서대문구,남가좌1동,서울특별시 서대문구 수색로4가길 6,126.914896,37.570475,남가좌1동%20엄마맛집,https://m.place.naver.com/restaurant/13018229


In [9]:
CH = dong_list[0]
CH.to_csv("url포함_1.csv", mode='w')

In [7]:
df = pd.read_csv('url포함_1.csv', sep=',', encoding = 'utf-8-sig')

In [5]:
df.head()

,Unnamed: 0,상호명,상권업종대분류명,상권업종소분류명,시도명,시군구명,행정동명,도로명주소,경도,위도,naver_keyword,naver_map_url
0,2634,감자바우곤드레밥,음식,한식/백반/한정식,서울특별시,서대문구,남가좌1동,서울특별시 서대문구 수색로2길 7,126.915967,37.569179,남가좌1동%20감자바우곤드레밥,https://m.place.naver.com/restaurant/13270081
1,7709,단비,음식,호프/맥주,서울특별시,서대문구,남가좌1동,서울특별시 서대문구 수색로 42,126.914240,37.570433,남가좌1동%20단비,https://m.place.naver.com/restaurant/1575351305
2,8090,고봉민김밥인,음식,라면김밥분식,서울특별시,서대문구,남가좌1동,서울특별시 서대문구 가재울미래로 2,126.917828,37.574368,남가좌1동%20고봉민김밥인,https://m.place.naver.com/restaurant/37793363
3,14688,광풍정육식당,음식,갈비/삼겹살,서울특별시,서대문구,남가좌1동,서울특별시 서대문구 수색로4가길 14-4,126.914553,37.570671,남가좌1동%20광풍정육식당,https://m.place.naver.com/restaurant/32731448
4,20094,엄마맛집,음식,한식/백반/한정식,서울특별시,서대문구,남가좌1동,서울특별시 서대문구 수색로4가길 6,126.914896,37.570475,남가좌1동%20엄마맛집,https://m.place.naver.com/restaurant/13018229


In [16]:
!pip install tqdm
from tqdm import tqdm_notebook
%config IPCompleter.greedy=True

In [17]:
visitor_review_num_list = []
visitor_review_rate_list = []
review_text_list = []
visitor_rate_list=[]
visitor_review_cont_list=[]
naver_map_star_review_stars_list = []
naver_map_star_review_qty_list = []
chromedriver = 'C:/Users/ChaeWon/Documents/3-1/파이썬과데이터분석/crawling/chromedriver.exe' 
import requests
from bs4 import BeautifulSoup

In [18]:
url = df['naver_map_url'][1]
req = requests.get(url + '/review/visitor')
req.encoding = 'utf-8'
source = req.text
soup = BeautifulSoup(source, 'html.parser')
driver.get(url + '/review/visitor')
time.sleep(2)

In [19]:
try:
    visitor_review_rate = driver.find_element_by_css_selector("#app-root > div > div > div.place_detail_wrapper > div:nth-child(1) > div._3uUKd > div._37n49 > span._1Y6hi._1A8_M > em").text
    print('별점: ',visitor_review_rate)#app-root > div > div > div > div.place_section.GCwOh > div._3uUKd > div._37n49 > span._1Y6hi._1A8_M

except Exception as e:
    if e == NoSuchElementException:
        print(e)

<ipython-input-19-928a0a50006c>:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  visitor_review_rate = driver.find_element_by_css_selector("#app-root > div > div > div.place_detail_wrapper > div:nth-child(1) > div._3uUKd > div._37n49 > span._1Y6hi._1A8_M > em").text


In [20]:
while True:
    try:
        driver.find_element(By.CLASS_NAME, "_3iTUo").click()
        time.sleep(1)
    except:
        break

In [21]:
a = driver.find_elements(By.CLASS_NAME, "WoYOw")
a
review_text = []
for i in a:
    review_text.append(i.text)
print(type(review_text))
print(review_text)

<class 'list'>
[]


## 수정 내용
- 더 보기 버튼을 클릭하는 줄을 넣었습니다 -> 더보기를 클릭하면서 저절로 스크롤이 되기 때문에 무한 스크롤 코드는 제거했습니다
- 검색결과가 없을 경우에 오류가 나는 것을 한꺼번에 처리하지 않고 각각으로 나누었습니다.
- -> try except 가 3번 반복되는 부분입니다. -> 그래서 각각 오류 발생시마다 null을 list에 추가 가능하게 되었습니다.
- 리뷰 본문 저장 시 soup의 find 함수로 하면 일부밖에 수집되지 않아서 selenium의 find_elements로 찾아 리스트에 저장했습니다.

## 오류 때문에 수정한 내용
- 중간에 크롬 드라이버가 제대로 작동되지 않는 부분이 생겨 requests.get부분에 headers를 추가했습니다.

## 새로 생긴 문제
- 이걸 돌리다가 중간에 닭내장집?에서 리뷰 수집이 멈추던데 오류가 뜨지도 않고 그냥 멈춰서 돌아가기만 하는 거라 문제가 무엇인지 모릅니다.
- 그래서 차선책으로 50개씩 나눠서 리뷰를 수집하면 어떨까.. 싶습니다. 아니면 제 컴퓨터에서만 안 돌아가는 건지..

In [ ]:
visitor_review_num_list = []
visitor_review_rate_list = []
visitor_text_list = []
#chromedriver = 'C:/Users/ChaeWon/Documents/3-1/파이썬과데이터분석/crawling/chromedriver.exe' 
driver = webdriver.Chrome(chromedriver, chrome_options=chrome_options) 
import requests
from bs4 import BeautifulSoup

for i, url in enumerate(tqdm_notebook(df['naver_map_url'])):
    req = requests.get(url + '/review/visitor', headers = {
"User-Agent" : "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36"
})
    ### 중간에 오류가 나길래 headers를 추가해줬습니다.
    req.encoding = 'utf-8'
    source = req.text
    soup = BeautifulSoup(source, 'html.parser')
    driver.get(url + '/review/visitor')
    time.sleep(2)


    # 간단 정보 가져오기
    store_name = driver.find_element_by_css_selector("#_title > span._3XamX").text
    print("_____________________________")
    print('<',store_name,'>')
    
    try:
        # 방문자 리뷰 수
        visitor_review_num = driver.find_element_by_css_selector("#app-root > div > div > div.place_detail_wrapper > div:nth-child(1) > div._3uUKd > div._37n49 > span:nth-child(2) > a > em").text
        print('방문자 리뷰 수: ',visitor_review_num)
        visitor_review_num_list.append(visitor_review_num)
    except:
        print(f"{i}행 리뷰수 문제가 발생")
        visitor_review_num_list.append('null')
        
    try:    
        # 방문자 별점 점수
        visitor_review_rate = driver.find_element_by_css_selector("#app-root > div > div > div.place_detail_wrapper > div:nth-child(1) > div._3uUKd > div._37n49 > span._1Y6hi._1A8_M > em").text
        print('별점: ',visitor_review_rate)
        visitor_review_rate_list.append(visitor_review_rate)
    except:
        print(f"{i}행 별점 문제가 발생")
        visitor_review_rate_list.append('null')
        
    
    while True:
        # 버튼을 찾아서 1초마다 계속 클릭
        try:
            driver.find_element(By.CLASS_NAME, "_3iTUo").click()
            time.sleep(1)
        except: # 못 찾으면 루프 빠져나감
            break
                
    # 끝까지 내렸으니 리뷰 수집해서 리스트에 넣기
    review_text_lists = driver.find_elements(By.CLASS_NAME, "WoYOw")
    if review_text_lists == []:
        review_text = 'null'
    else:
        review_text = []
        for i in review_text_lists:
            review_text.append(i.text)
            
    print(type(review_text))
    print(review_text)
    visitor_text_list.append(review_text)
        
driver.quit()

df['naver_visitor_review_num'] = visitor_review_num_list  
df['naver_visitor_review_rate'] = visitor_review_rate_list
df['naver_visitor_review_text'] = visitor_text_list

<ipython-input-23-69780825d897>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver, chrome_options=chrome_options)
<ipython-input-23-69780825d897>:5: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chromedriver, chrome_options=chrome_options)
<ipython-input-23-69780825d897>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, url in enumerate(tqdm_notebook(df['naver_map_url'])):


  0%|          | 0/94 [00:00<?, ?it/s]

<ipython-input-23-69780825d897>:22: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  store_name = driver.find_element_by_css_selector("#_title > span._3XamX").text
<ipython-input-23-69780825d897>:28: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  visitor_review_num = driver.find_element_by_css_selector("#app-root > div > div > div.place_detail_wrapper > div:nth-child(1) > div._3uUKd > div._37n49 > span:nth-child(2) > a > em").text
<ipython-input-23-69780825d897>:37: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  visitor_review_rate = driver.find_element_by_css_selector("#app-root > div > div > div.place_detail_wrapper > div:nth-child(1) > div._3uUKd > div._37n49 > span._1Y6hi._1A8_M > em").text


_____________________________
< 감자바우 곤드레밥 >
0행 리뷰수 문제가 발생
0행 별점 문제가 발생
<class 'list'>
['밥 왜이렇게 늦게먹냐고 이럴줄 알았으면 밥 안 줬다고 주인이 역정내는집. 7시에 주문- 7시 20분에 밥나옴 - 식사 2 0분째부터 사장이 그릇치우기 시작하더니 밥 왜이리 늦게 먹냐 나 퇴근해야된다. 이럴줄알았음 밥 안줬다(진짜 이 렇게말함)정색하고 꾸짖음ㅋㅋ 기분 잡쳐서 걍 그자리에서 계산하니까 왜 남기냐고 다 먹고가라고함. ㅋㅌㅋ...', '대체로 신선해요', '곤드레나물밥에 청국장 밑반찬이 좋아요!! 맛나요', '곤드레밥 먹고싶을때 종종 찾습니다. 반찬종류도 많고 양도 넉넉히 나오네요~ 개인적으로 곤드레가 좀더 많이 들어가면 좋겠고 반찬이 덜 짜면 좋겠어요~ 그래도 맛나고 배부르게 잘 먹었어요^^', '맛나요', '신선해요', '굿', '너무맛있게 잘먹었어요!', '곤드레돌솥밥도 맛나지만 나오는 반찬도 엄마손맛처럼 맛있답니다.', '맛나요', '굿', '💙', '굿', '음식이 진짜맛있고 정성스런 느낌입니다^^', '배부르게 먹고왔네요!!', '친절하고 맛있어요', '곤드레밥이 들솓밥으로 줘서 좋고 청국장도 짜지만 구수한게 맛나고요..다른 반찬들은 종류는 많은데 맛은그냥 보통인듯, 밥이 조금 부족한데 추가 공기밥이 없는건 많이 아숩네요', '가격이 올랐어요.1인 12.000원.. 나물이 많아서 좋은데 생선이나 고기가 없어서 비싸게 느껴지네요. 쌈은 뭐에 싸먹으라고 주셨는지 의아했고요..위생 부분도 그렇고 코로나 방역수칙이 제대로 지켜지지않아서 다신 가고싶지 않네요..', '맛있어요', '맛있어요', '굿', '굿', '맛있어요', '넘맛잇어요', '오래된 맛집인데 오늘 처음알았네요. 사장님 혼자 하시는데 재촉하는 손님도 없고 기다리면 순서대로 다 갖다주시네요. 청국장 특히 맛있고요 밑반찬 가짓수에놀라고 먹어보면 맛에놀랍니다', '엄마가 차려준 맛있는 밥상^^', '조아요', '좋아요', '건강한 집밥

_____________________________
< 역전주야식당 >
5행 리뷰수 문제가 발생
5행 별점 문제가 발생
<class 'list'>
['돈가스 진짜 맛있습니다.', '좋아요', '맛있습니다 !!', '좋아요', '바지락 칼국수 먹으러 들어갔다가 한잔하고 나와야하는집', '굿', '네조아용']
_____________________________
< 풍년떡집 >
6행 리뷰수 문제가 발생
6행 별점 문제가 발생
<class 'list'>
['맛있어요', '친절해요', '떡이 맛있어여', '좋아요!', '좋아요', '떡이맛있는집', '맛있어여']
_____________________________
< 피카소 >
7행 리뷰수 문제가 발생
7행 별점 문제가 발생
<class 'str'>
null
_____________________________
< 든든김밥 서대문구본점 >
8행 리뷰수 문제가 발생
8행 별점 문제가 발생
<class 'list'>
['예전에 이화사랑 김밥을 좋아해서 오랜만에 방문했어요. 전화하다가 끊겼는데도 다 준비해주셔서 감사했어요:) 너무 바쁘셔서 마음의 여유가 없으신지 엄청 친절하신건 아니였지만 그래도 괜찮았어요❤️ 김밥은 조금 비싸지만 추억의 맛이 있어서 좋아요 감사합니다😊', '맛있고 친절하세요!', '이화여대에 있던 맛집입니다. 여전히 예전 맛 그대로 유지하고 있어서 좋습니다.', '굿', '굿', '친절, 깔끔, 양 많아요 ^^', '맛있는 음식 감사합니다!', '기본김밥은 없고 4천원부터 시작해요 요즘 김밥값이 많이 올랐네요ㅠ', '크고 맛있긴한데 김밥이 너무 비싸요', '굿', '굿', '김밥 간이 쎕니다.매장은 넓고 쾌적!', '굿', '굿', '가격대비 특별하진 않고 그냥 평범해요.', '맛있어요!!', '별로예요', '김밥도 맛있지만 든든라면 진짜 마성의 맛!', '갱차나여', '가격이 500원씩 올랐어요. 김밥은 비싼데 실제보면 좀 부실해요...', '주말에 간간히 가는데 항상 배달주문때문에 바쁘시더라구요. 

_____________________________
< 예천식당 >
10행 리뷰수 문제가 발생
10행 별점 문제가 발생
<class 'list'>
['여기 사장님이 친절하시고 제육볶음이 맛나요', '맛나요', '굿', '굿', '가성비 좋아요 맛도 좋아요', '맛나요', '굿', '그닥', '여사장님이 마스크 착용을 안하셨네요. 친절하신대 마스크는 좀 써주세요.', '맛있어요']
_____________________________
< 산해정 >
11행 리뷰수 문제가 발생
11행 별점 문제가 발생
<class 'list'>
['오래만에 방문했는데 여전히 한결같고 맛있네요..^^ 식당에 모든메뉴가 다 맛있기 힘든법인데 이 집은 다 맛있는집.. 특히 내장탕은 맛도 최고 가성비도 최고.. 곁들여 나오는 반찬도 다 정갈하고 맛있습니다.. 특히나 김치들은 양손 엄지척을 부릅니다..저 대파김치 그리고 쪽파김치 때문에 삼겹살을 시켜먹게도 만든답니다..ㅎ', '여기 삼겹살 레알 찐임.. (3주동안 3번감) 보기에 외관은 음..? 하지만 안에는 꽤 넓음 회식하러 갔는데 입맛 까다로운 대표님도 회식 다음날 또 갔다고 하심ㅋㅋ 여사장님도 친절하심 또 또 갈거임', '맛있고 친절해요', '자주가는곳 설렁탕 최애메뉴 반찬이 잘 바뀌는데 다 맛있음 오늘은 파김치 짱맛', '역시나 맛 좋은 삼겹살과 묵은지!!!!!', '위생에 신경좀 써주셨으면..', '크흐~~ 무슨 말이 더 필요할까요 산해정인데!!!', '내장탕이 된장베이스로 독특하고 고소하고 맛있어요 설렁탕은 내 최애 메뉴 반찬에 자부심있으신데 다 맛있어서 좋아요', '오늘도 맛있게 먹으며 모임하고 왔어요', '삼겹살 볶음밥 밑반찬 맛집!!', '굿', '좋아요', '내장탕 우주대존맛 ♥️', '굿', '좋아요', '고기 후 볶음밥 1등!!!!', '돼지갈비 맛있고 사장님 친절합니다 다만, 내부 청소는 좀 신경쓰시면 좋겠어요 먼지와 때낀 선풍기들이 식욕을 조금 떨어뜨렸네요ㅠ', 'ㄷ', '굿', '깨끗했음해요', '내장탕 맛있어요

_____________________________
< 카페카티지 >
20행 리뷰수 문제가 발생
20행 별점 문제가 발생
<class 'list'>
['사장님 직원분 모두 친절하시고 인테리어도 감각적! 커피맛은 정말 맛있어요 :-)', '알바생언니가 너무 친절하세요❣️ 여기 크림라떼 너무좋아하는데 순두부처럼 부들부들하고 첫입먹으면 진실의미간을 볼수있어요,, 디저트 특히 초코나무숲이랑 블루베리크럼블 모두 존맛이고 가좌에 이런 분위기의 디저트맛집이 있어서 행복합니다아 앞으로 순두부라떼 계속 맛있게만들어주세용 헤헤', '동네에서 이런카페 참 좋네요 커피도 완전 맛있구요 ~~두번째방문인데 또 올거에요 지인데리고 ~', '카피와 브라우니가 맛있어서 자주 들립니다. 인테리어가 깔끔하고 좋습니다.', '사장님 친절하시고 커피 맛있어요 디저트도 최고👍🏻❤️❤️', '밀크티가 너무 맛있습니다. 자주방문하는데 사장님도 친절하십니다', '커피가 맛있어요', '매장이 너무 깔끔하고 이뻐요😊 직원분들도 친절하셔서 방문만으로도 힐링되네요😆 커피도 너무 맛있고 메뉴설명도 꼼꼼하게 잘해주셔서 커피 초보에게는 너무 좋네요☺', '커피 맛있어요~!!', '굿', '손글씨로 happy new year 적힌 🍪 서비스로 주셨는데 너무 귀여워요 😍 새해첫날 따뜻한 핫초코도 너무 좋네요❤', '좋아요', '인테리어도 너무 예쁘고 음료 또한 너무 맛있네요!!! 동네에서 단연 최고카페!!', '좋아요', '굿', '좋아요', '좋아요', '당근케익맛집', '좋아요', '좋아요', '굳', '좋아요.', '좋아요', '좋아요.', '맛있어요', '좋아요', '스콘이랑 카페랑 너무 맛있어요! 이것저것 먹고싶어서 계속 추가주문하게 되네요 인테리어도 너무 예뻐요 이런카 페 생겨서 너무 좋습니다! 직원분이 추천해주시는 커피가 제일 맛있었어요', '기본에 충실한 곳!!', '좋아요', '좋아요', '좋아요.', '좋아요', '커피와 디저트 너무 맛있고 분위기도 좋아요!', '좋아요', '좋아요', '좋아요', 

_____________________________
< 새벽집 통닭 >
24행 리뷰수 문제가 발생
24행 별점 문제가 발생
<class 'list'>
['최고♡', '젛아요', '오늘은 닭이 좀 부드럽기보단 퍽퍽살이 심하게 퍽퍽해서 좀 남겼네요~ㅜ 다음부턴 치킨무 잘 챙겨주세요>< 잘 먹엇습니다~', '미리 전화주문하고 가니 좋아요', '좋아요', '고소하고 맛있어요~!!', '누룽지통닭 맛있어요!', '퍽퍽살 마저 부드러워서 두돌 안 된 아기도 함께 잘 먹었어요! 누룽지도 맛있고 겨자 소스도 달달하니 매력적이네요ㅋㅋ', '요즘 자주가게되네여ㅋㅋ 부모님이 광팬인곳👍👍👍👍👍 맛있어여^^~사장님도 친절하시구요^^ 찐단골이 되엇 네여^^', '맛있어요!!누룽지가 좀 더 많으면 좋을거같아요', '부모님과 항상 맛잇게. 먹는곳~맛나요', '누릉지가맛있어요', '진짜 맛있어요', '좋아요', '진짜 맛있어여👍👍👍👍👍 보통 먹는 닭들과는 달라요~', '항상 맛나요👍👍👍👍👍', '소리부터 맛있는데 촉촉한 통닭과 빠삭노릇한 누룽지한입이면 맥주가 그냥😮 거기에 시원한 무생채까지 먹으면 완벽한조합 사이좋으신 부부 두분이서 운영하시는데 정말 친절하시고 매번 맛있게먹고갑니다 ㅎㅎ', '말해뭐해요 ㅎㅎㅎ 👍', '존맛 동네맛집', '부부사장님 두분다 친절하고 맛은말해뭐해요~~~🍗🐔🐤😁👍👍', '맛있어요~', '누룽지통닭이라고 하면 서오릉 가는줄 알았는데 새벽집이 훨씬 더 맛있네요~ 골뱅이나 계란찜도 너무 맛있었어요 사장님 부부 너무 친절하시고 인상도 좋으시구요~~ 재방문 의사 10000프로입니다~ 기분좋게 너무 잘 먹었습니다~', '담백 한 것은 전기 통닭 달달하며 마늘을 좋아한다면 마늘 누룽지 전기 통닭. 먹어본 누룽지 통닭중에 가장 맛있습 니다. 자주 배달시켜먹을게요.', '누룽지전기통닭 강추합니다^^', '맛있어요', '좋아요', '기름이 쫙 빠진 닭구이. 누룽지 맛이 일품입니다!!!', '맛있어요', '누룽지통닭 별미예요~ 정말 담백하고 고소합니다!', '맛있는데 누룽지 양이

_____________________________
< 카페 모래내 1번지 >
28행 리뷰수 문제가 발생
28행 별점 문제가 발생
<class 'list'>
['크크 모래내 1번지 언제나 사랑입니다 애정입니다💕 코로나 때문에 한동안 문 닫아서 너무 슬펐습니다ㅠㅠ 이 동네에서 정말 최고 최애 카페에요ㅠㅠ 커피도 맛있는데 사장님도 엄청 친절하시고 공간이 아늑하니 좋습니당♡♡ 나만 알고픈 카페!!', '내 취업기간을 책임져준 카페 스*벅스보다 그린티 라떼가 맛있다 사장님이 커피에 자부심이 강해 가끔 뭐라뭐라 설 명은 해주시는데 솔직히 무슨말인지 1도 모르겠다; 머리로는 이해가 안되도 입에서 맛있으니 합격 무엇보다 아늑하 고 분위기가 깡패라 데이트장소로도 좋은듯 나중에 여자친구 생기면(언젠가) 같이 와야지... #모래내 #심야...', '카페가 예쁘고 집중이 잘됨여~~ 이 동네에서 늦게까지도 하고 분위기도 강추욜', '여기는 진짜 제 인생 카페 ㅠㅠ 카공이나 애인이랑 데이트할 때 여기만 가욥 후하❣️ 조용하고 분위기가 다 한 카페 입니다 ㅠ 음료, 디저트 맛이랑 가격 둘 다 착하기 힘든데, 여긴 그걸 해내네요 ㅠㅠ 이런데가 가까운데 있어서 너 무 좋아용❣️', '아늑하고좋아요', '가좌 최고 심야카페 ㅇㅈ 이만한 카페가 우리 동네에 잏어서 진짜 좋다고 생각함 ㅠㅠ', '일요일엔 오전11시부터 오픈해서 좋아요 토요일도 오전부터 오픈하면 좋겠다는 바램이,, 대부분 혼자오는 사람들 이 많고 간혹 단체손님도 오는데 전반적으로 조용하고 좋습니다 커피값도 저렴해서 좋구요 원래 케이크가 있었는 데 지금 리뉴얼중이네여 ㅠㅠ 케이크 있어서 참 좋았는데.. 얼른 다시 케이크가 메뉴에 생기길 바랍니당..', '저으ㅣ 최애카패 사장님도 너무 좋으시고 커피맛도 좋습니다 ㅎㅎ', '맛있어용 이동네에서 제일 좋은 카페같아용 번창하세요 사장님🥳🥳', '평일에도 아침부터 열었으면 좋겠어요 ㅠㅠ 분위기 너무 좋고.. 공부도 잘되오', '커피가 저렴해서 좋아요~ㅎㅎ', 'ㄱ']
___________

In [35]:
len(visitor_review_num_list)

50

In [45]:
df_1['naver_visitor_review_text'].replace('[]','null')

0     [밥 왜이렇게 늦게먹냐고 이럴줄 알았으면 밥 안 줬다고 주인이 역정내는집. 7시에 ...
1                                                    []
2     [맛있어요 김밥, 늦게까지 영업해서 좋아요, 좋아요, 쏘, 좋아요, 좋아요 또 올께...
3                                                    []
4     [22년만에 다시 방문했다...어렸을 적 그 공간이 시간과 함께 멈춰있다. 그리웠던...
5                 [바지락 칼국수 먹으러 들어갔다가 한잔하고 나와야하는집, 네조아용]
6              [친절해요, 떡이 맛있어여, 좋아요!, 좋아요, 떡이맛있는집, 맛있어여]
7                                                    []
8     [예전에 이화사랑 김밥을 좋아해서 오랜만에 방문했어요. 전화하다가 끊겼는데도 다 준...
9     [텀블러 이용해서 자주 가는 스벅입니다. 작업할 수 있는 테이블이 길게 있어서 유용...
10    [가성비 좋아요 맛도 좋아요, 맛나요, 굿, 그닥, 여사장님이 마스크 착용을 안하셨...
11    [오래만에 방문했는데 여전히 한결같고 맛있네요..^^ 식당에 모든메뉴가 다 맛있기 ...
12                           [삼십분 서비스준다고하고 돈 받아요 가지마세요]
13    [닭살이랑 알밥이 너무 맛있어요! 두개 더 시켜먹었어여 ㅎㅎ, 음식이 맛있어요, 좋...
14    [빨리 포장해주셔서 바로 찾아왔어요 !! 오렌지에이드 맛있네요. 친구는 체리콕매니아...
15                                                   []
16    [점심 A셋트 시켯어요~ 우선 음식 너무 맛잇네요~ 이동네에서 쫄면 맛은 최고인듯합...
17    [친절해요, 친절해요, 좋아요, 사장님이 늘 친절하셔서 좋아요, 엔진오일 갈

In [46]:
df_1['naver_visitor_review_text']

0     [밥 왜이렇게 늦게먹냐고 이럴줄 알았으면 밥 안 줬다고 주인이 역정내는집. 7시에 ...
1                                                    []
2     [맛있어요 김밥, 늦게까지 영업해서 좋아요, 좋아요, 쏘, 좋아요, 좋아요 또 올께...
3                                                    []
4     [22년만에 다시 방문했다...어렸을 적 그 공간이 시간과 함께 멈춰있다. 그리웠던...
5                 [바지락 칼국수 먹으러 들어갔다가 한잔하고 나와야하는집, 네조아용]
6              [친절해요, 떡이 맛있어여, 좋아요!, 좋아요, 떡이맛있는집, 맛있어여]
7                                                    []
8     [예전에 이화사랑 김밥을 좋아해서 오랜만에 방문했어요. 전화하다가 끊겼는데도 다 준...
9     [텀블러 이용해서 자주 가는 스벅입니다. 작업할 수 있는 테이블이 길게 있어서 유용...
10    [가성비 좋아요 맛도 좋아요, 맛나요, 굿, 그닥, 여사장님이 마스크 착용을 안하셨...
11    [오래만에 방문했는데 여전히 한결같고 맛있네요..^^ 식당에 모든메뉴가 다 맛있기 ...
12                           [삼십분 서비스준다고하고 돈 받아요 가지마세요]
13    [닭살이랑 알밥이 너무 맛있어요! 두개 더 시켜먹었어여 ㅎㅎ, 음식이 맛있어요, 좋...
14    [빨리 포장해주셔서 바로 찾아왔어요 !! 오렌지에이드 맛있네요. 친구는 체리콕매니아...
15                                                   []
16    [점심 A셋트 시켯어요~ 우선 음식 너무 맛잇네요~ 이동네에서 쫄면 맛은 최고인듯합...
17    [친절해요, 친절해요, 좋아요, 사장님이 늘 친절하셔서 좋아요, 엔진오일 갈

In [47]:
df_1.to_csv("naver_result.csv", encoding="utf-8-sig")